# 04e - Risk Stratification Models

**Objetivo**: Desarrollar modelos de estratificación de riesgo para identificar subgrupos de pacientes y fenotipos específicos de Alzheimer
 
**Componentes principales**:
- Modelos de clustering para identificación de fenotipos
- Estratificación por características demográficas y genéticas
- Modelos jerárquicos para diferentes niveles de riesgo
- Análisis de subgrupos específicos (APOE, edad, biomarkers)


---

## Importar librerías y Configurar ML Flow

In [ ]:
import sys
import os
sys.path.append('../scripts/modeling')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
import mlflow
import mlflow.sklearn
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Configuración de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Librerías importadas correctamente")
print(f"📅 Fecha de ejecución: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


In [ ]:
# Configuración MLflow
mlflow.set_experiment("alzheimer_risk_stratification")

def log_clustering_metrics(model_name, labels, X, n_clusters=None):
    """Registra métricas de clustering en MLflow"""
    with mlflow.start_run(nested=True):
        mlflow.set_tag("model_type", "clustering")
        mlflow.set_tag("algorithm", model_name)
        
        if n_clusters:
            mlflow.log_param("n_clusters", n_clusters)
        
        # Métricas de clustering
        silhouette = silhouette_score(X, labels)
        calinski = calinski_harabasz_score(X, labels)
        
        mlflow.log_metric("silhouette_score", silhouette)
        mlflow.log_metric("calinski_harabasz_score", calinski)
        mlflow.log_metric("unique_clusters", len(np.unique(labels)))
        
        return silhouette, calinski

print("🔧 MLflow configurado para Risk Stratification")

## Cargar datos procesados y Análisis inicial

In [ ]:
try:
    df = pd.read_csv('../data/processed/integrated_features_final.csv')
    print(f"📊 Dataset cargado: {df.shape}")
    print(f"📈 Score de riesgo disponible: {df['composite_risk_score'].notna().sum()} registros")
    print(f"🎯 Distribución de categorías:")
    print(df['risk_category'].value_counts())
except FileNotFoundError:
    print("❌ Error: Archivo de features no encontrado")
    print("💡 Asegúrate de ejecutar el notebook 03_feature_engineering_master.ipynb primero")


In [ ]:
# Análisis inicial de distribución de riesgo
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Distribución del score compuesto
axes[0,0].hist(df['composite_risk_score'].dropna(), bins=50, alpha=0.7, color='skyblue')
axes[0,0].set_title('Distribución Score de Riesgo Compuesto')
axes[0,0].set_xlabel('Score de Riesgo')
axes[0,0].set_ylabel('Frecuencia')

# Distribución por categorías
df['risk_category'].value_counts().plot(kind='bar', ax=axes[0,1], color='lightcoral')
axes[0,1].set_title('Distribución por Categorías de Riesgo')
axes[0,1].set_xlabel('Categoría de Riesgo')
axes[0,1].tick_params(axis='x', rotation=45)

# Score por APOE status
if 'APOE_e4_carrier' in df.columns:
    df.boxplot(column='composite_risk_score', by='APOE_e4_carrier', ax=axes[1,0])
    axes[1,0].set_title('Score de Riesgo por APOE Status')
    axes[1,0].set_xlabel('APOE e4 Carrier')

# Score por edad (si disponible)
if 'age_standardized' in df.columns:
    axes[1,1].scatter(df['age_standardized'], df['composite_risk_score'], alpha=0.5)
    axes[1,1].set_title('Score de Riesgo vs Edad Estandarizada')
    axes[1,1].set_xlabel('Edad Estandarizada')
    axes[1,1].set_ylabel('Score de Riesgo')

plt.tight_layout()
plt.show()

print("📊 Análisis descriptivo completado")

## Preparación de datos para clustering

In [ ]:
# Seleccionar features clave para estratificación
stratification_features = []

# Features genéticas
genetic_features = [col for col in df.columns if 'APOE' in col or 'genetic' in col.lower()]
stratification_features.extend(genetic_features[:5])  # Top 5

# Features de biomarkers
biomarker_features = [col for col in df.columns if any(x in col.upper() for x in ['ABETA', 'TAU', 'biomarker'])]
stratification_features.extend(biomarker_features[:5])  # Top 5

# Features cognitivas
cognitive_features = [col for col in df.columns if any(x in col.upper() for x in ['CDRSB', 'DIAGNOSIS'])]
stratification_features.extend(cognitive_features[:3])  # Top 3

# Features de actividad/sueño
activity_features = [col for col in df.columns if 'sleep' in col.lower() or 'activity' in col.lower()]
stratification_features.extend(activity_features[:4])  # Top 4

# Features demográficas
demo_features = [col for col in df.columns if any(x in col.lower() for x in ['age', 'gender'])]
stratification_features.extend(demo_features[:3])  # Top 3

# Remover duplicados y filtrar features existentes
stratification_features = list(set(stratification_features))
stratification_features = [f for f in stratification_features if f in df.columns]
stratification_features.append('composite_risk_score')  # Incluir target

print(f"🎯 Features seleccionadas para estratificación: {len(stratification_features)}")

In [ ]:
print("📋 Features principales:")
for f in stratification_features[:10]:
    print(f"   • {f}")
if len(stratification_features) > 10:
    print(f"   ... y {len(stratification_features)-10} más")


In [ ]:
# Preparar datos para clustering
df_clustering = df[stratification_features].copy()
df_clustering = df_clustering.dropna()

print(f"📊 Dataset para clustering: {df_clustering.shape}")
print(f"🎯 Registros válidos: {len(df_clustering)}")

## Separar features y target y Estandarizar features

In [ ]:
# Separar features y target
X_clustering = df_clustering.drop('composite_risk_score', axis=1)
y_risk_score = df_clustering['composite_risk_score']

# Estandarizar features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_clustering)

print("✅ Datos preparados y estandarizados para clustering")

## 1. K-Means Clustering para identificar fenotipos

In [ ]:
# 1. K-Means Clustering para identificar fenotipos
with mlflow.start_run(run_name="phenotype_identification"):
    mlflow.set_tag("phase", "risk_stratification")
    mlflow.set_tag("approach", "phenotype_clustering")
    
    # Determinar número óptimo de clusters
    silhouette_scores = []
    calinski_scores = []
    k_range = range(2, 8)
    
    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        labels = kmeans.fit_predict(X_scaled)
        
        sil_score = silhouette_score(X_scaled, labels)
        cal_score = calinski_harabasz_score(X_scaled, labels)
        
        silhouette_scores.append(sil_score)
        calinski_scores.append(cal_score)
    
    # Seleccionar k óptimo
    optimal_k = k_range[np.argmax(silhouette_scores)]
    
    # Modelo final K-Means
    kmeans_final = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
    phenotype_labels = kmeans_final.fit_predict(X_scaled)
    
    # Registrar métricas
    mlflow.log_param("optimal_k", optimal_k)
    mlflow.log_metric("best_silhouette", max(silhouette_scores))
    mlflow.log_metric("best_calinski", calinski_scores[np.argmax(silhouette_scores)])
    mlflow.sklearn.log_model(kmeans_final, "kmeans_phenotypes")
    
    print(f"🎯 Fenotipos identificados: {optimal_k} clusters")
    print(f"📊 Silhouette Score: {max(silhouette_scores):.3f}")



In [ ]:
# Análisis de fenotipos identificados
df_clustering['phenotype'] = phenotype_labels

# Análisis por fenotipo
phenotype_analysis = df_clustering.groupby('phenotype').agg({
    'composite_risk_score': ['mean', 'std', 'count']
}).round(3)

print("🧬 ANÁLISIS DE FENOTIPOS IDENTIFICADOS")
print("=" * 50)
print(phenotype_analysis)

# Visualización de fenotipos
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# PCA para visualización
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Scatter plot de fenotipos en espacio PCA
scatter = axes[0,0].scatter(X_pca[:, 0], X_pca[:, 1], c=phenotype_labels, cmap='tab10', alpha=0.6)
axes[0,0].set_title('Fenotipos en Espacio PCA')
axes[0,0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} varianza)')
axes[0,0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} varianza)')
plt.colorbar(scatter, ax=axes[0,0])

# Distribución de score por fenotipo
df_clustering.boxplot(column='composite_risk_score', by='phenotype', ax=axes[0,1])
axes[0,1].set_title('Score de Riesgo por Fenotipo')
axes[0,1].set_xlabel('Fenotipo')

# Conteo de fenotipos
phenotype_counts = df_clustering['phenotype'].value_counts().sort_index()
axes[1,0].bar(phenotype_counts.index, phenotype_counts.values, color='lightgreen')
axes[1,0].set_title('Distribución de Fenotipos')
axes[1,0].set_xlabel('Fenotipo')
axes[1,0].set_ylabel('Número de Pacientes')

# Score promedio por fenotipo
phenotype_means = df_clustering.groupby('phenotype')['composite_risk_score'].mean()
axes[1,1].bar(phenotype_means.index, phenotype_means.values, color='salmon')
axes[1,1].set_title('Score Promedio por Fenotipo')
axes[1,1].set_xlabel('Fenotipo')
axes[1,1].set_ylabel('Score Promedio')

plt.tight_layout()
plt.show()

## 2. Estratificación por APOE Status

In [ ]:
# 2. Estratificación por APOE Status
if 'APOE_e4_carrier' in df.columns:
    with mlflow.start_run(run_name="apoe_stratification"):
        mlflow.set_tag("stratification_type", "genetic_apoe")
        
        # Análisis por APOE status
        apoe_analysis = df.groupby('APOE_e4_carrier').agg({
            'composite_risk_score': ['mean', 'std', 'count', 'median']
        }).round(3)
        
        print("\n🧬 ESTRATIFICACIÓN POR APOE STATUS")
        print("=" * 40)
        print(apoe_analysis)
        
        # Modelos específicos por APOE status
        apoe_carriers = df[df['APOE_e4_carrier'] == 1]
        apoe_non_carriers = df[df['APOE_e4_carrier'] == 0]
        
        mlflow.log_metric("apoe_carriers_count", len(apoe_carriers))
        mlflow.log_metric("apoe_non_carriers_count", len(apoe_non_carriers))
        mlflow.log_metric("apoe_risk_difference", 
                         apoe_carriers['composite_risk_score'].mean() - 
                         apoe_non_carriers['composite_risk_score'].mean())
        
        print(f"📊 APOE4 Carriers: {len(apoe_carriers)} ({len(apoe_carriers)/len(df)*100:.1f}%)")
        print(f"📊 Non-carriers: {len(apoe_non_carriers)} ({len(apoe_non_carriers)/len(df)*100:.1f}%)")


## 3. Estratificación por Edad

In [ ]:
# 3. Estratificación por Edad
if 'age_standardized' in df.columns:
    with mlflow.start_run(run_name="age_stratification"):
        mlflow.set_tag("stratification_type", "demographic_age")
        
        # Crear grupos de edad
        df['age_group'] = pd.cut(df['age_standardized'], 
                                bins=[-np.inf, -1, 0, 1, np.inf], 
                                labels=['Very_Low', 'Low', 'High', 'Very_High'])
        
        age_analysis = df.groupby('age_group').agg({
            'composite_risk_score': ['mean', 'std', 'count']
        }).round(3)
        
        print("\n👥 ESTRATIFICACIÓN POR EDAD")
        print("=" * 35)
        print(age_analysis)
        
        # Registrar métricas
        for group in df['age_group'].unique():
            if pd.notna(group):
                group_data = df[df['age_group'] == group]
                mlflow.log_metric(f"age_group_{group}_mean_risk", 
                                group_data['composite_risk_score'].mean())
                mlflow.log_metric(f"age_group_{group}_count", len(group_data))


## 4. Gaussian Mixture Model para estratificación probabilística

In [ ]:
# 4. Gaussian Mixture Model para estratificación probabilística
with mlflow.start_run(run_name="probabilistic_stratification"):
    mlflow.set_tag("model_type", "gaussian_mixture")
    
    # Determinar número óptimo de componentes
    n_components_range = range(2, 6)
    bic_scores = []
    aic_scores = []
    
    for n_comp in n_components_range:
        gmm = GaussianMixture(n_components=n_comp, random_state=42)
        gmm.fit(X_scaled)
        bic_scores.append(gmm.bic(X_scaled))
        aic_scores.append(gmm.aic(X_scaled))
    
    # Seleccionar mejor modelo
    optimal_components = n_components_range[np.argmin(bic_scores)]
    
    # Modelo GMM final
    gmm_final = GaussianMixture(n_components=optimal_components, random_state=42)
    gmm_final.fit(X_scaled)
    
    # Asignaciones probabilísticas
    risk_probabilities = gmm_final.predict_proba(X_scaled)
    risk_assignments = gmm_final.predict(X_scaled)
    
    # Registrar modelo
    mlflow.log_param("n_components", optimal_components)
    mlflow.log_metric("best_bic", min(bic_scores))
    mlflow.log_metric("best_aic", min(aic_scores))
    mlflow.sklearn.log_model(gmm_final, "gmm_risk_stratification")
    
    print(f"🎯 Componentes GMM óptimos: {optimal_components}")
    print(f"📊 BIC Score: {min(bic_scores):.1f}")

In [ ]:
# Análisis de componentes GMM
df_clustering['gmm_component'] = risk_assignments
df_clustering['gmm_max_prob'] = np.max(risk_probabilities, axis=1)

gmm_analysis = df_clustering.groupby('gmm_component').agg({
    'composite_risk_score': ['mean', 'std', 'count'],
    'gmm_max_prob': 'mean'
}).round(3)

print("\n🎲 ANÁLISIS DE COMPONENTES GMM")
print("=" * 40)
print(gmm_analysis)

# Visualización GMM
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Componentes en espacio PCA
scatter = axes[0].scatter(X_pca[:, 0], X_pca[:, 1], c=risk_assignments, 
                         cmap='viridis', alpha=0.6)
axes[0].set_title('Componentes GMM en Espacio PCA')
axes[0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%})')
axes[0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%})')
plt.colorbar(scatter, ax=axes[0])

# Probabilidades máximas
axes[1].hist(df_clustering['gmm_max_prob'], bins=30, alpha=0.7, color='orange')
axes[1].set_title('Distribución de Probabilidades Máximas GMM')
axes[1].set_xlabel('Probabilidad Máxima')
axes[1].set_ylabel('Frecuencia')

plt.tight_layout()
plt.show()

## 5. Modelo jerárquico de estratificación

In [ ]:
# 5. Modelo jerárquico de estratificación
with mlflow.start_run(run_name="hierarchical_stratification"):
    mlflow.set_tag("model_type", "hierarchical")
    
    # Crear estratificación jerárquica
    # Nivel 1: Riesgo general (bajo, moderado, alto)
    df_hierarchical = df.copy()
    
    # Nivel 2: Subestratos por características específicas
    conditions = []
    
    # Alto riesgo + APOE4
    if 'APOE_e4_carrier' in df.columns:
        high_risk_apoe = (df['risk_category'] == 'High') & (df['APOE_e4_carrier'] == 1)
        conditions.append(('High_Risk_APOE4', high_risk_apoe))
    
    # Alto riesgo sin APOE4
    if 'APOE_e4_carrier' in df.columns:
        high_risk_no_apoe = (df['risk_category'] == 'High') & (df['APOE_e4_carrier'] == 0)
        conditions.append(('High_Risk_No_APOE4', high_risk_no_apoe))
    
    # Riesgo moderado con biomarcadores altos
    if 'biomarker_risk_score' in df.columns:
        mod_risk_high_bio = (df['risk_category'] == 'Moderate') & \
                           (df['biomarker_risk_score'] > df['biomarker_risk_score'].quantile(0.75))
        conditions.append(('Moderate_Risk_High_Biomarkers', mod_risk_high_bio))
    
    # Asignar subestratos
    df_hierarchical['substrata'] = 'Other'
    for name, condition in conditions:
        df_hierarchical.loc[condition, 'substrata'] = name
    
    # Análisis de subestratos
    substrata_analysis = df_hierarchical.groupby('substrata').agg({
        'composite_risk_score': ['mean', 'std', 'count']
    }).round(3)
    
    print("\n🏗️ ESTRATIFICACIÓN JERÁRQUICA")
    print("=" * 40)
    print(substrata_analysis)
    
    # Registrar métricas
    for substrata in df_hierarchical['substrata'].unique():
        substrata_data = df_hierarchical[df_hierarchical['substrata'] == substrata]
        mlflow.log_metric(f"substrata_{substrata}_count", len(substrata_data))
        mlflow.log_metric(f"substrata_{substrata}_mean_risk", 
                         substrata_data['composite_risk_score'].mean())

## 6. Modelo de estratificación basado en biomarcadores

In [ ]:
# 6. Modelo de estratificación basado en biomarcadores
if any('tau' in col.lower() for col in df.columns) or any('abeta' in col.lower() for col in df.columns):
    with mlflow.start_run(run_name="biomarker_stratification"):
        mlflow.set_tag("stratification_type", "biomarker_based")
        
        # Seleccionar features de biomarcadores
        biomarker_cols = [col for col in df.columns if 
                         any(marker in col.lower() for marker in ['tau', 'abeta', 'ptau'])]
        
        if biomarker_cols:
            biomarker_data = df[biomarker_cols + ['composite_risk_score']].dropna()
            
            # Clustering específico para biomarcadores
            if len(biomarker_data) > 100:
                X_bio = biomarker_data.drop('composite_risk_score', axis=1)
                X_bio_scaled = StandardScaler().fit_transform(X_bio)
                
                # K-means para biomarcadores
                kmeans_bio = KMeans(n_clusters=3, random_state=42)
                bio_clusters = kmeans_bio.fit_predict(X_bio_scaled)
                
                biomarker_data['biomarker_cluster'] = bio_clusters
                
                bio_cluster_analysis = biomarker_data.groupby('biomarker_cluster').agg({
                    'composite_risk_score': ['mean', 'std', 'count']
                }).round(3)
                
                print("\n🧪 ESTRATIFICACIÓN POR BIOMARCADORES")
                print("=" * 45)
                print(bio_cluster_analysis)
                
                # Registrar métricas
                mlflow.log_param("biomarker_features", len(biomarker_cols))
                mlflow.log_metric("biomarker_samples", len(biomarker_data))
                mlflow.sklearn.log_model(kmeans_bio, "biomarker_clustering")

## Resumen final de estratificación

In [ ]:
print("\n" + "="*60)
print("📊 RESUMEN DE ESTRATIFICACIÓN DE RIESGO")
print("="*60)

stratification_summary = {
    'Total_Pacientes': len(df),
    'Fenotipos_Identificados': optimal_k,
    'Componentes_GMM': optimal_components,
}

if 'APOE_e4_carrier' in df.columns:
    stratification_summary['APOE4_Carriers'] = df['APOE_e4_carrier'].sum()
    stratification_summary['APOE4_Percentage'] = f"{df['APOE_e4_carrier'].mean()*100:.1f}%"

if 'age_group' in df.columns:
    stratification_summary['Grupos_Edad'] = df['age_group'].nunique()

if 'substrata' in locals():
    stratification_summary['Subestratos_Jerarquicos'] = df_hierarchical['substrata'].nunique()

for key, value in stratification_summary.items():
    print(f"🎯 {key}: {value}")

print("\n✅ Estratificación de riesgo completada exitosamente")
print("📁 Modelos guardados en MLflow")
print("🔄 Listo para la siguiente fase: Evaluación y Optimización")

## Guardar resultados de estratificación

In [ ]:
# Guardar resultados de estratificación
results_to_save = df_clustering.copy()
if 'age_group' in df.columns:
    results_to_save['age_group'] = df['age_group']
if 'substrata' in locals():
    results_to_save['substrata'] = df_hierarchical['substrata']

results_to_save.to_csv('../data/processed/risk_stratification_results.csv', index=False)
print("📁 Resultados guardados en: ../data/processed/risk_stratification_results.csv")

---

__Abraham Tartalos__